In [1]:
import torchvision.datasets as datasets
import numpy as np 
import os
from functools import partial
from jax import jit
import jax 
import jax.numpy as jnp
import torchvision
import jax.random as random
import matplotlib.pyplot as plt
import ott

/home/tordjx/miniconda3/envs/rapidsjpp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
from typing import Any
from jax.numpy import ndarray
from ott.geometry.costs import TICost

def sigma(z) :
    jnp.arcsinh(z)


#maybe regti should herit from ticost

@jax.tree_util.register_pytree_node_class
class RegTICost(TICost) : 
    def __init__(self, cost = lambda x : 0.5*jnp.sum(x**2),reg_name ="l1",gamma=0.1 ) :
        super().__init__()
        self.reg_name = reg_name
        self.cost = cost
        if self.reg_name not in ['l1','stvs' ,None]:
            raise Exception("Norm name must be one of l1, stvs, None")
        self.gamma = gamma
    def reg(self,z) :
        if self.reg_name == "l1" :
            return self.gamma * jnp.sum(abs(z))
        elif self.reg_name == "stvs" : 
            return self.gamma * jnp.sum(self.gamma ** 2 * (sigma(z) + 0.5 -0.5 *jnp.exp(-2*sigma(z))))
        else :
            return 0
    def h(self, z) :
        return self.cost(z) + self.reg(z)

    def prox_reg(self,z) : #Would be needed to compute the entropic map for general h
        if self.reg_name == "l1" :
            return jnp.where((1-self.gamma/abs(z))>0,(1-self.gamma/abs(z)),0) * z
        elif self.reg_name == "stvs" :
            return jnp.where((1-self.gamma**2/(abs(z)**2))>0,(1-self.gamma**2/(abs(z)**2)),0) * z
        else :
            return 0
    def h_legendre(self, z: jnp.ndarray) -> float:  # noqa: D102
        q = self.prox_reg(jax.lax.stop_gradient(z))
        return jnp.sum(q * z) - self.h(q)



In [2]:
import scanpy
path_file = "/home/tordjx/Downloads/Massively multiplex chemical transcriptomics at single-cell resolution - K562.h5ad"
data= scanpy.read_h5ad(path_file)
#Remove cells with less than 200 expressed genes
df = data.obs[data.obs['num_genes_expressed']>200]
#Remove genes expressed in less than 20 cells
import numpy as np
ann_data = data[data.obs.index.isin(df.index)]
column_sums = ann_data.X.sum(axis=0)
column_sums_dense = np.asarray(column_sums).flatten()
mask = column_sums_dense > 20
filtered_ann_data = ann_data[:, mask]
#Select the 5 drugs Belinostat, Dacinostat, Givinostat, Hesperadin, and Quisinostat, and the control
medocs = ["Belinostat (PXD101)",
"Hesperadin",
"Givinostat (ITF2357)",
"Dacinostat (LAQ824)",
"Quisinostat (JNJ-26481585) 2HCl", "Vehicle"]
scanpy.pp.normalize_total(filtered_ann_data)
scanpy.pp.log1p(filtered_ann_data)
scanpy.pp.highly_variable_genes(filtered_ann_data, n_top_genes=5000)
scanpy.tl.rank_genes_groups(filtered_ann_data, groupby='product_name', method='t-test')
perturbation_marker_names = { k : filtered_ann_data.uns['rank_genes_groups']['names'][k][:50] for k in medocs}
#Whole set 
cell_names = {k : filtered_ann_data.obs[k == df["product_name"]].index for k in medocs}
cell_data_full = { k : filtered_ann_data[filtered_ann_data.obs.index.isin(cell_names[k])] for k in medocs}
cell_marker_index = {k : [list(cell_data_full[k].to_df().columns).index(item) for item in perturbation_marker_names[k]] for k in medocs}
cell_data_full = { k : np.asarray(cell_data_full[k].X.todense()) for k in medocs}
cell_data_marker = {k : cell_data_full[k][:, cell_marker_index[k]] for k in medocs}
#Top 5k
top5k_filtered_ann_data = filtered_ann_data[:, filtered_ann_data.var['highly_variable']]
cell_names = {k : top5k_filtered_ann_data.obs[k == df["product_name"]].index for k in medocs}
cell_data_5k = { k : top5k_filtered_ann_data[top5k_filtered_ann_data.obs.index.isin(cell_names[k])] for k in medocs}
cell_data_5k = { k : np.asarray(cell_data_5k[k].X.todense()) for k in medocs}

/home/tordjx/miniconda3/envs/rapidsjpp/lib/python3.10/site-packages/scanpy/preprocessing/_normalization.py:206: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/tordjx/miniconda3/envs/rapidsjpp/lib/python3.10/site-packages/scanpy/tools/_rank_genes_groups.py:429: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, "names"] = self.var_names[global_indices]
/home/tordjx/miniconda3/envs/rapidsjpp/lib/python3.10/site-packages/scanpy/tools/_rank_genes_groups.py:431: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame

In [19]:
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from ott.tools.sinkhorn_divergence import sinkhorn_divergence

def experiment(medoc, cell_data, cell_data_marker,reg_name, pca_bool = False, gamma_list =[2**-i  for i in range(5)]):
    X = cell_data['Vehicle']
    y = cell_data[medoc]
    Xfolds = KFold(n_splits=10, shuffle=True, random_state=42).split(X)
    yfolds=  KFold(n_splits=10, shuffle=True, random_state=42).split(y)
    result = []
    cost = RegTICost(reg_name=reg_name)
    for fold, ((train_index_X, test_index_X), (train_index_y, test_index_y)) in enumerate(zip(Xfolds, yfolds)):
        X_train, X_test = X[train_index_X], X[test_index_X]
        y_train, y_test = y[train_index_y], y[test_index_y]
        if pca_bool :
            pca = PCA(n_components=50).fit(X_train)
            X_train,y_train = pca.transform(X_train), pca.transform(y_train)
            X_test,y_test = pca.transform(X_test), pca.transform(y_test)
            dic_gamma ={}
            for gamma in gamma_list: 
                epsilon=0.1
                geom = ott.geometry.pointcloud.PointCloud(X_train, y_train, cost_fn=cost,scale_cost = "mean")
                problem = ott.problems.linear.linear_problem.LinearProblem(geom)
                solver = ott.solvers.linear.sinkhorn.Sinkhorn(threshold=epsilon)
                out = solver(problem)
                y_test_pred = out.to_dual_potentials().transport(X_test)
                dic_gamma[gamma] =  sinkhorn_divergence(ott.geometry.pointcloud.PointCloud, y_test_pred, y_test).divergence
                """COMPUTE Ranked biased overlap (Webber et al., 2010) with p = 0.9,
                between the 50 perturbation marker genes as computed
                on all data with SCANPY, and the following per-gene
                statistic, computed using a map as follows: average (on
                fold) expression of (predicted) perturbed cells from original control cells (this tracks changes in log-expression
                before/after predicted treatment)"""
                """COMPUTE Coefficient of determination (R2
                ) between the average
                ground-truth / predicted gene expression on the 50 perturbation markers (Lotfollahi et al., 2019)"""
            result.append(dic_gamma)
    return result

In [20]:
experiment(medocs[0], cell_data_5k, None, pca_bool=True)

[Array(16.46344, dtype=float32),
 Array(15.376888, dtype=float32),
 Array(14.863703, dtype=float32),
 Array(15.006366, dtype=float32),
 Array(15.618315, dtype=float32),
 Array(15.302305, dtype=float32),
 Array(15.599243, dtype=float32),
 Array(15.5090885, dtype=float32),
 Array(15.460455, dtype=float32),
 Array(16.38553, dtype=float32)]

False

In [206]:
X_p.shape

(1024, 2)